Asynchronous parallel processing of FVS keyfiles

In [1]:
import os
import glob
import subprocess
import ipyparallel as ipp
import shutil

A function to execute FVS that will be mapped to all keyfiles.

In [3]:
def run_fvs(keyfile):
    fvs_exe = 'C:\\FVSbin\\'+os.path.split(keyfile)[-1][:5]+'.exe'
    subprocess.call([fvs_exe, '--keywordfile='+keyfile]) # run fvs
    
    base_dir = os.path.split(keyfile)[0]
    base_name = os.path.split(keyfile)[-1].split('.')[0]
    
    # clean-up the outputs
    # move the .out and .key file
    path = os.path.join(base_dir, 'completed','keyfiles')
    if not os.path.exists(path): 
        os.makedirs(path)
    shutil.move(keyfile, os.path.join(base_dir,'completed','keyfiles'))
    path = os.path.join(base_dir, 'completed','outfiles')
    if not os.path.exists(path):
        os.makedirs(path)
    shutil.move(os.path.join(base_dir,base_name+'.out'), os.path.join(base_dir,'completed','outfiles'))
    
    # delete the other files
    os.remove(os.path.join(base_dir, base_name+'.trl'))
    return keyfile

Run the following command in a command prompt to start up a cluster of workers:

`>> activate Py3.5 # or other environment name`

`(Py3.5)>> ipcluster start -n 4 # or other number of cores`

In [43]:
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [ ]:
# if you want to run a single keyfile, use this
# subprocess.call(['C:\\FVSbin\\FVSpn.exe', '--keywordfile=C:\\GitHub\\FSC_Case_Studies\\keyfiles_to_run\\PN\\fvsPN_stand1_rx4_off0.key'])

Create a direct view of the workers and a load-balanced view for submitting jobs

In [44]:
dv = c[:] # direct view
v = c.load_balanced_view() # load-balanced view

# import packages to all workers
with dv.sync_imports():
    import subprocess
    import shutil
    import os

Execute an ayschronous batch of FVS runs for all the keyfiles

In [46]:
# gather the list of keyfiles to run
run_dir = os.path.abspath('keyfiles_to_run\\PN')
to_run = glob.glob(os.path.join(run_dir, '*.key'))
print('{:,}'.format(len(to_run)), 'keyfiles found.')
res = v.map_async(run_fvs, to_run)
print('Started batch processing.')

39,001 keyfiles found.
Started batch processing.


Monitor progress of batch run

In [ ]:
# Default method
# res.wait_interactive(10)

# OR USE A PROGRESS BAR!
from tqdm import tqdm_notebook
import time

num_done = res.progress
with tqdm_notebook(total=len(res), initial=num_done, desc='FVS Run Progress', unit='keyfile') as pbar:
    while not res.ready():
        increment = res.progress - num_done
        pbar.update(increment)
        num_done += increment
        time.sleep(5) # update progress bar every 5 seconds

In [40]:
# Return a true/false if full set of jobs completed
# res.ready()

# Cancels the batch
res.abort()

In [ ]:
print('Human time spent:', res.wall_time)
print('Computer time spent:', res.serial_time)
print('Async speedup:', res.serial_time/res.wall_time)
print('Human time per FVS run:', res.wall_time/res.progress)
print('Computer time per FVS run:', res.serial_time/res.progress)

In [41]:
c.shutdown(hub=True)